In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.clustering import KMeans
from pyspark.sql.functions import when

# Utilisation de la base de données Hive, et charge les données de la table

In [2]:
spark = SparkSession.builder \
    .appName("Categorie") \
    .config("spark.hadoop.hive.metastore.uris", "thrift://hive-metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()


In [3]:
spark.sql("USE concessionnaire")
client_immat_df = spark.sql("SELECT * FROM client_immatriculation_merged")

In [4]:
client_immat_df.show(10, truncate=False)

+---------------+---+-----+----+-------------------+---------------+---------------+-------+---------+-----------+--------+--------+-------+--------+-------+------+
|immatriculation|age|sexe |taux|situationfamilliale|nbenfantacharge|deuxiemevoiture|marque |puissance|longueur   |nbplaces|nbportes|couleur|occasion|prix   |modele|
+---------------+---+-----+----+-------------------+---------------+---------------+-------+---------+-----------+--------+--------+-------+--------+-------+------+
|0 OG 13        |56 |Homme|528 |Célibataire        |0              |false          |Audi   |75       |courte     |5       |5       |blanc  |true    |12817.0|A2    |
|0 OZ 65        |32 |Homme|563 |En Couple          |2              |false          |Ford   |125      |longue     |5       |5       |blanc  |false   |23900.0|Mondeo|
|1 PD 24        |28 |Homme|230 |En Couple          |2              |false          |Renault|245      |très longue|5       |5       |blanc  |true    |34440.0|Vel   |
|10 EW 37 

# Définis les catégories.

In [5]:

client_immat_df = client_immat_df.withColumn(
    "categorie",
    when((col("puissance") <= 100) & (col("longueur") == "courte"), "citadine")
    .when((col("puissance") > 100) & (col("puissance") <= 200) & (col("longueur") == "courte") & (col("prix") < 30000), "citadine premium")
    .when((col("puissance") > 100) & (col("puissance") <= 200) & (col("longueur").isin("moyenne", "longue")), "familiale")
    .when((col("puissance") > 200) & (col("prix") > 30000), "sportive")
    .when((col("nbplaces") >= 7), "monospace")
    .when((col("prix") > 50000) & col("marque").isin("BMW", "Mercedes", "Jaguar", "Audi", "Porsche"), "luxe")
    .otherwise("autre")
)


# Affiche le resultat 

In [6]:
client_immat_df.select("marque","modele","categorie","prix","longueur","puissance").show(30, truncate=False)

+--------+------+----------------+--------+-----------+---------+
|marque  |modele|categorie       |prix    |longueur   |puissance|
+--------+------+----------------+--------+-----------+---------+
|Audi    |A2    |citadine        |12817.0 |courte     |75       |
|Ford    |Mondeo|familiale       |23900.0 |longue     |125      |
|Renault |Vel   |sportive        |34440.0 |très longue|245      |
|Jaguar  |X-Type|familiale       |37100.0 |longue     |197      |
|Jaguar  |X-Type|familiale       |37100.0 |longue     |197      |
|Volvo   |S80   |sportive        |50500.0 |très longue|272      |
|Audi    |A2    |citadine        |18310.0 |courte     |75       |
|Mini    |Copper|citadine premium|18200.0 |courte     |115      |
|Ford    |Mondeo|familiale       |23900.0 |longue     |125      |
|BMW     |M5    |sportive        |94800.0 |très longue|507      |
|Renault |Vel   |sportive        |49200.0 |très longue|245      |
|Peugeot |1007  |citadine        |13750.0 |courte     |75       |
|Audi    |

# Enregistre en base de données

In [7]:
client_immat_df.write.mode("overwrite").saveAsTable("client_immatriculation_categorie")

In [1]:
category_counts = client_immat_df.groupBy("categorie").count().orderBy("categorie").collect()
categories = [row["categorie"] for row in category_counts]
counts = [row["count"] for row in category_counts]

plt.figure(figsize=(10, 6))
sns.barplot(x=categories, y=counts)
plt.xlabel("Catégorie")
plt.ylabel("Nombre de véhicules")
plt.title("Distribution des véhicules par catégorie")
plt.show()

NameError: name 'client_immat_df' is not defined